<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion6EvaluationKeyWordsV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terminology extraction with keywords, association measures, etc.

Here we enrich extracted MWEs with keyness information (and other paremeters) and test Precision / Recall

## Preparing gold standard annotation dictinaries

In [ ]:
# Stage 1: Preparing Gold standard: Reading / extracting information from gold standard: creating a list of annotated terms
# set 1 (same text annotated by two annotators)
!wget https://heibox.uni-heidelberg.de/f/ae1110c4f9ad42b9a3d5/?dl=1
!mv index.html?dl=1 BGH1_s00Astghik.txt
!wget https://heibox.uni-heidelberg.de/f/398e7a10fa3241519f26/?dl=1
!mv index.html?dl=1 BGH1_s00Maia.txt

# set 2 (same text annotated by two annotators)
!wget https://heibox.uni-heidelberg.de/f/0c787f26123f49178639/?dl=1
!mv index.html?dl=1 BGH2_s00Hayk.txt
!wget https://heibox.uni-heidelberg.de/f/356205b502fb4d759ad5/?dl=1
!mv index.html?dl=1 BGH2_s00Nino.txt

# set 3 (same text annotated by two annotators)
!wget https://heibox.uni-heidelberg.de/f/ed0c7af9a9d04967b449/?dl=1
!mv index.html?dl=1 BGH3_s00Tamar.txt
# !wget 
# !mv index.html?dl=1 

In [5]:
# one more will be added: Frau Khachatryan
!cat BGH1_s00Astghik.txt BGH1_s00Maia.txt BGH2_s00Hayk.txt BGH2_s00Nino.txt BGH3_s00Tamar.txt >BGH0_s00GoldStandard.txt

In [11]:
# a useful function for recording / visualising current stage of dictionaries
def printDictionary(DictionaryFrq, FOut, K = 1, Rev = True): # printing a dictionary: by values or alphabetically
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[K], reverse=Rev):
        FOut.write(Word + '\t' + str(Frq) + '\n')
    return

In [15]:
FInput = open('BGH0_s00GoldStandard.txt', 'r')
FOutput = open('BGH0_s01GoldS_Terms.txt', 'w')
# for statistical purposes - separately single and multiword terms
FOutputDict1w = open('BGH0_s01GoldS_D1w.txt', 'w') # 1-word terms
FOutputDict2w = open('BGH0_s01GoldS_D2w.txt', 'w') # 2-word terminological expressions
FOutputDict3w = open('BGH0_s01GoldS_D3w.txt', 'w') # 3-word terminological expressions
FOutputDictMWE = open('BGH0_s01GoldS_DMWE.txt', 'w') # more than 3 words

In [16]:
# creating gold-standard dictionaries for evaluation tasks:
import re, os, sys
LGSTerms = [] # gold standard terms
DGS1w = {} # dictionary of single words
DGS2w = {} # dictionary of 2-word expressions
DGS3w = {} # dictionary of 3-word expressions
DGSMWE = {} # dictionary of other mwes
IGS1w = 0 # number of annotated tokens of single words
IGS2w = 0
IGS3w = 0
IGSMWE = 0 # number of annotated tokens of multiwords
for SLine in FInput:
    LAnnotatedTermsInLine = re.findall('<<([^><]+)>>', SLine)
    LGSTerms.extend(LAnnotatedTermsInLine)

for GSTerm in LGSTerms:
    GSTerm = GSTerm.strip()
    GSTerm = GSTerm.strip('„“"().')

    # everything is converted to upper case for quick dictionary lookup
    GSTerm = GSTerm.upper()
    
    GSTerm = re.sub(' +', ' ', GSTerm)
    LGSTErms = re.split(' ', GSTerm)
    if len(LGSTErms) > 3:
        IGSMWE += 1
        try: DGSMWE[GSTerm] += 1
        except: DGSMWE[GSTerm] = 1
    elif len(LGSTErms) > 2:
        IGS3w += 1
        try: DGS3w[GSTerm] += 1
        except: DGS3w[GSTerm] = 1
    elif len(LGSTErms) > 1:
        IGS2w += 1
        try: DGS2w[GSTerm] +=1
        except: DGS2w[GSTerm] = 1
    else:
        IGS1w += 1
        try: DGS1w[GSTerm] +=1
        except: DGS1w[GSTerm] = 1

    FOutput.write(GSTerm + '\n')

FOutputDictMWE.write('# Number of tokens: ' + str(IGSMWE) + '\n')
FOutputDict3w.write('# Number of tokens: ' + str(IGS3w) + '\n')
FOutputDict2w.write('# Number of tokens: ' + str(IGS2w) + '\n')
FOutputDict1w.write('# Number of tokens: ' + str(IGS1w) + '\n')

printDictionary(DGSMWE, FOutputDictMWE)
printDictionary(DGS3w, FOutputDict3w)
printDictionary(DGS2w, FOutputDict2w)
printDictionary(DGS1w, FOutputDict1w)

FOutputDictMWE.flush()
FOutputDictMWE.close()
FOutputDict3w.flush()
FOutputDict3w.close()
FOutputDict2w.flush()
FOutputDict2w.close()
FOutputDict1w.flush()
FOutputDict1w.close()

FOutput.flush()
FOutput.close()

FInput.close()

## Preparing the 'keyness' dictionary

In [17]:
!wget https://heibox.uni-heidelberg.de/f/aa4560e627bd4b1d8055/?dl=1
!mv index.html?dl=1 TK_KW_Verif_V02.csv

--2021-10-27 09:36:48--  https://heibox.uni-heidelberg.de/f/aa4560e627bd4b1d8055/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/1f73a3b5-80d0-4fb7-aa0f-f9d08aabc629/TK_KW_Verif_V02.csv [following]
--2021-10-27 09:36:49--  https://heibox.uni-heidelberg.de/seafhttp/files/1f73a3b5-80d0-4fb7-aa0f-f9d08aabc629/TK_KW_Verif_V02.csv
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 38812 (38K) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  37.90K  90.7KB/s    in 0.4s    

2021-10-27 09:36:50 (90.7 KB/s) - ‘index.html?dl=1’ saved [38812/38812]



In [19]:
# Stage 2: Preparing a dictionnary of keyness weights
FInputKW = open('TK_KW_Verif_V02.csv', 'r')
FOutputKW = open('TK_KW_Verif_V02.txt', 'w')
# FOutputGSKWS1w = open('BGH1_s01GoldSKW_D1w.txt', 'w')
# FOutputGSKWS2w = open('BGH1_s01GoldSKW_D2w.txt', 'w')
# FOutputGSKWS3w = open('BGH1_s01GoldSKW_D3w.txt', 'w')
# FOutputGSKWSMWE = open('BGH1_s01GoldSKW_MWE.txt', 'w')

In [20]:
DScoresKW = {} # keywords - scores
DScoresNK = {} # non-keywords
DStatKW = {} # status: key/non-key-word
for Line in FInputKW:
    LFieldsKW = re.split('\t', Line)
    SWord = LFieldsKW[1]
    AKScore = float(LFieldsKW[2])
    AKStat = float(LFieldsKW[3])
    DStatKW[SWord] = AKStat
    if AKStat > 0: # change value to 0.5 if we need to restrict to 'sure' terms only (value 1)
        DScoresKW[SWord] = AKScore
    else:
        DScoresNK[SWord] = AKScore

In [21]:
# we create a dictionary of keyness values with only upper case letters, which will be checked against also uppercased term candidates
# the same dictionary as DScoresKW, but with ensured conversion in to upper case:
DScoresREKWquick = {} # dictionary of RE
for kw, val in DScoresKW.items():
    SUpperC = kw.upper() # making sure our key words are in upper case
    # these are the alternatives, which we do not consider in this stage...
    # SLowerC = kw.lower()
    # SSentenceC = kw.capitalize()
    # RPatternKW = re.compile('^' + kw + '$', re.IGNORECASE)
    DScoresREKWquick[SUpperC] = val # 
    # DScoresREKWquick[SLowerC] = val # 
    # DScoresREKWquick[SSentenceC] = val # 

In [22]:
printDictionary(DScoresREKWquick, FOutputKW)